<a href="https://colab.research.google.com/github/evanjholt/D1V1-Lobbying/blob/main/D1V1_Lobbying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import ipywidgets as widgets  # <-- CORRECTED IMPORT
from IPython.display import display, clear_output

# --- Function to load all data from the web ---
def load_all_lobbying_data():
    zip_url = 'https://lobbycanada.gc.ca/media/zwcjycef/registrations_enregistrements_ocl_cal.zip'
    print(f"-> Downloading data from: {zip_url}")
    try:
        zip_response = requests.get(zip_url)
        zip_response.raise_for_status()
        print("✅ Download complete.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading ZIP file: {e}")
        return None

    print("-> Extracting all CSV files into DataFrames...")
    dataframes = {}
    with ZipFile(BytesIO(zip_response.content)) as zip_file:
        for file_name in zip_file.namelist():
            if file_name.endswith('.csv'):
                with zip_file.open(file_name) as csv_file:
                    df = pd.read_csv(csv_file, encoding='latin1', low_memory=False)
                    dataframes[file_name] = df
    print("✅ All datasets loaded into memory.")
    return dataframes

# --- Create the interactive explorer ---
def create_explorer(dataframes):
    table_names = list(dataframes.keys())

    # Create a dropdown widget using the correct library
    dropdown = widgets.Dropdown(
        options=table_names,
        description='Select a table to preview:',
        style={'description_width': 'initial'},
        layout={'width': 'max-content'}
    )

    # Function to display table info when dropdown changes
    def on_change(change):
        if change['type'] == 'change' and change['new']:
            table_name = change['new']
            df = dataframes[table_name]

            # Clear previous output and display new info
            with out:
                clear_output(wait=True)
                print(f"\n--- Previewing '{table_name}' ---")
                print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")

                print("\n[Columns and Data Types]")
                df.info(verbose=False) # Concise info

                print("\n[First 10 Rows]")
                display(df.head(10))

    dropdown.observe(on_change, names='value')

    # Display the dropdown and the output area
    out = widgets.Output()
    display(dropdown, out)

    # Trigger the display for the first item initially
    on_change({'type': 'change', 'new': table_names[0]})

# --- Main Execution ---
all_data = load_all_lobbying_data()
if all_data:
    create_explorer(all_data)
else:
    print("Could not load data. Please try running the cell again.")

-> Downloading data from: https://lobbycanada.gc.ca/media/zwcjycef/registrations_enregistrements_ocl_cal.zip
✅ Download complete.
-> Extracting all CSV files into DataFrames...
✅ All datasets loaded into memory.


Dropdown(description='Select a table to preview:', layout=Layout(width='max-content'), options=('Registration_…

Output()